In [1]:
from biobert_embedding.embedding import BiobertEmbedding

/Users/ariellubonja/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Original pip install biobert-embedding does not work (it's too old)

I forked the repo. Install using `pip install git+https://github.com/ariellubonja/biobert_embedding`

In [2]:
## Example 1
text = "Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis."\

# Class Initialization (You can set default 'model_path=None' as your finetuned BERT model path while Initialization)
biobert = BiobertEmbedding()

Using existing models/pytorch_model.bin
Using existing models/config.json
Using existing models/vocab.txt


#### Basic example to make sure it works

In [3]:
word_embeddings = biobert.word_vector(text)
sentence_embedding = biobert.sentence_vector(text)

print("Text Tokens: ", biobert.tokens)
# Text Tokens:  ['breast', 'cancers', 'with', 'her2', 'amplification', 'have', 'a', 'higher', 'risk', 'of', 'cns', 'metastasis', 'and', 'poorer', 'prognosis', '.']

print ('Shape of Word Embeddings: %d x %d' % (len(word_embeddings), len(word_embeddings[0])))
# Shape of Word Embeddings: 16 x 768

print("Shape of Sentence Embedding = ",len(sentence_embedding))
# Shape of Sentence Embedding =  768

## Example 2
sentence_vector1 = biobert.sentence_vector('Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis.')
sentence_vector2 = biobert.sentence_vector('Breast cancers with HER2 amplification are more aggressive, have a higher risk of CNS metastasis, and poorer prognosis.')

Text Tokens:  ['breast', 'cancers', 'with', 'her2', 'amplification', 'have', 'a', 'higher', 'risk', 'of', 'cns', 'metastasis', 'and', 'poorer', 'prognosis', '.']
Shape of Word Embeddings: 16 x 768
Shape of Sentence Embedding =  768


In [5]:
from scipy.spatial.distance import cosine as cosine_distance

cosine_sim = 1 - cosine_distance(sentence_vector1, sentence_vector2)
print('cosine similarity:', cosine_sim)
#cosine similarity: 0.992756187915802

cosine similarity: 0.9927560091018677


In [6]:
help(biobert.word_vector)

Help on method word_vector in module biobert_embedding.embedding:

word_vector(text, handle_oov=True, filter_extra_tokens=True) method of biobert_embedding.embedding.BiobertEmbedding instance



### Word Embeddings of Drugs.com drug names (without supporting text)

Find the drug names I've crawled [here](https://drive.google.com/drive/folders/1EO659a-tyjfXjKzHk-M0WreZBR14MgRM?usp=sharing)

In [ ]:
import pandas as pd

# Specify the file path of the CSV file
csv_file_path = 'path/to/your/file.csv'

# Load the CSV file into a Pandas DataFrame
dataframe = pd.read_csv(csv_file_path)

# Now you can work with the loaded data in the DataFrame
# For example, you can display the first few rows using the head() function
print(dataframe.head())


## TODO try ClinicalBERT https://github.com/EmilyAlsentzer/clinicalBERT

In [8]:
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


## TODO try PubChem Embeddings, DrugBERT